# Задание 1
Полезные ссылки:
- https://en.wikipedia.org/wiki/Linear_regression
- https://en.wikipedia.org/wiki/Least_squares#Solving_the_least_squares_problem


In [157]:
import numpy as np
import pandas as pd
import scipy
from math import sqrt
from IPython.display import display, Markdown

df = pd.read_csv('data/lin-reg.csv')
df.head()

,Output,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,108.997,0.334,-92.123,14.909,-43.866,24.876,-57.008,-51.184,-96.558,-17.006,20.042
1,911.877,-60.177,-19.648,14.792,81.555,99.005,-30.756,-12.662,-15.049,21.225,23.006
2,1063.620,88.101,22.545,40.254,-6.953,63.602,-99.745,-6.910,34.521,97.959,60.746
3,-146.979,29.961,-64.830,33.632,-44.687,61.911,42.770,-8.657,-39.879,-35.115,71.327
4,-670.859,22.997,82.418,33.892,-89.676,-92.545,95.573,85.631,68.880,83.979,-36.734


## (a)
Выкладки в `notes/1.jpg`

In [158]:
y = df['Output'].copy()
x = df.to_numpy().copy()
# Make a dummy column
x[:,0] = 1

n, k = x.shape

display(Markdown(f'''
$n={n}$ $k={k}$
'''))


$n=100$ $k=11$


In [159]:
# Calculate least squares estimation for c (c_hat)
e = np.identity(n)
x_t = x.transpose()
a = x_t @ x

try:
    a_inv = np.linalg.inv(a)
except np.LinAlgError:
    print('Cannot proceed with uninvertible matrix a :(')
    raise

c_hat = a_inv @ x_t @ y

delim = r'\\'
display(Markdown(f'''
ОНК $c$:
$$
\\hat{{c}}=
\\left(
    \\begin{{smallmatrix}}
        {delim.join(f'{c_i:.3}' for c_i in c_hat)}
    \\end{{smallmatrix}}
\\right)
$$
'''))

# Estimation of variance
tmp = y - x @ c_hat
s_sq = np.dot(tmp.transpose(), tmp)
sigma_sq = s_sq / (n - k)

display(Markdown(f'''
Оценка остаточной дисперсии: $$\\hat{{\\sigma^2}}={sigma_sq:.3f}$$
'''))


ОНК $c$:
$$
\hat{c}=
\left(
    \begin{smallmatrix}
        -0.173\\1.08\\1.96\\2.98\\3.97\\5.07\\-5.01\\4.11\\-3.02\\2.1\\-0.983
    \end{smallmatrix}
\right)
$$



Оценка остаточной дисперсии: $$\hat{\sigma^2}=852.003$$


## (b)
Выкладки в `notes/2.jpg`

In [160]:
# Find the confidence intervals for c_i
alpha = 0.05

# TODO: remove list comprehension?
tmp = np.array([sqrt(a_inv_ii * s_sq / (n - k)) for a_inv_ii in np.diag(a_inv)])
left_border = c_hat - tmp * scipy.stats.t.ppf(1 - alpha/2, n - k)
right_border = c_hat - tmp * scipy.stats.t.ppf(alpha/2, n - k)

intervals = zip(range(k), left_border, right_border)

display(Markdown(r'''
Доверительные интервалы для $c$: $${}$$
'''.format(delim.join(f'c_{{ {i} }} \\in [{lb:.3f}, {rb:.3f}]' for i, lb, rb in intervals))))

# Find confidence intervals for variance

left_border = s_sq / scipy.stats.chi2.ppf(1 - alpha/2, n - k)
right_border = s_sq / scipy.stats.chi2.ppf(alpha/2, n - k)

display(Markdown(f'''
Доверительный интервал для $\sigma^2$: $$\sigma^2 \in [{left_border:.3f}, {right_border:.3f}]$$
'''))


Доверительные интервалы для $c$: $$c_{ 0 } \in [-6.268, 5.922]\\c_{ 1 } \in [0.964, 1.187]\\c_{ 2 } \in [1.853, 2.061]\\c_{ 3 } \in [2.869, 3.098]\\c_{ 4 } \in [3.864, 4.075]\\c_{ 5 } \in [4.970, 5.163]\\c_{ 6 } \in [-5.112, -4.907]\\c_{ 7 } \in [4.003, 4.216]\\c_{ 8 } \in [-3.126, -2.911]\\c_{ 9 } \in [1.997, 2.196]\\c_{ 10 } \in [-1.091, -0.875]$$



Доверительный интервал для $\sigma^2$: $$\sigma^2 \in [648.165, 1170.309]$$


## (c)
Выкладки в `notes/3.jpg`

In [161]:
# Calculate statistic for t-test
t_stat = sqrt(n - k) * c_hat / np.sqrt(s_sq * np.diag(a_inv))

lb, rb = scipy.stats.t.ppf(alpha/2, n - k), scipy.stats.t.ppf(1-alpha/2, n - k)

# Test hypotheses via confidence interval and p-value
display(Markdown(f'Confidence interval for t-statistic is $[{lb:.3f}, {rb:.3f}]$'))
for i, stat in zip(range(k), t_stat):
    p_value = 2 * min(scipy.stats.t.cdf(stat, n - k), 1 - scipy.stats.t.cdf(stat, n - k))
    if lb <= stat <= rb:
        assert p_value >= alpha
        display(Markdown(f'Inside confidence interval: $t={stat:.3f}$, $p={p_value:.3f}$'))
    else:
        assert p_value <= alpha
        display(Markdown(f'Outside confidence interval: $t={stat:.3f}$, $p={p_value:.3f}$'))

Confidence interval for t-statistic is $[-1.987, 1.987]$

Inside confidence interval: $t=-0.057$, $p=0.955$

Outside confidence interval: $t=19.230$, $p=0.000$

Outside confidence interval: $t=37.324$, $p=0.000$

Outside confidence interval: $t=51.910$, $p=0.000$

...

Outside confidence interval: $t=-97.043$, $p=0.000$

Outside confidence interval: $t=76.596$, $p=0.000$

Outside confidence interval: $t=-55.719$, $p=0.000$

Outside confidence interval: $t=41.763$, $p=0.000$

Outside confidence interval: $t=-18.062$, $p=0.000$

## (d)
Выкладки в `notes/4.jpg`

In [164]:
m = 3
t = np.zeros((m, k))
t[0,3] = 1
t[1,8] = 1
t[2,10] = 1

d = t @ a_inv @ t.transpose()

try:
    d_inv = np.linalg.inv(d)
except np.LinAlgError:
    print('Cannot proceed with uninvertible matrix d :(')
    raise

qt = (t @ c_hat).transpose() @ d_inv @ t @ c_hat

f_stat = (n - k) / m * qt / s_sq

lb = scipy.stats.f.ppf(1 - alpha, m, n - k)

p_r = 1 - scipy.stats.f.cdf(f_stat, m, n - k)
if lb <= f_stat:
    assert p_r <= alpha
    display(Markdown(f'Hypothesis $H_0$ is denied (p-value is $p={p_r:.3f}$)'))
else:
    assert p_r >= alpha
    display(Markdown(f'Hypothesis $H_0$ is not denied (p-value is $p={p_r:.3f}$)'))

Hypothesis $H_0$ is denied (p-value is $p=0.000$)